# 1.Data Load

In [1]:
import warnings
warnings.filterwarnings('ignore')

# 라이브러리

import pandas as pd
import random
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import VotingClassifier
import lightgbm as lgbm
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier

In [2]:
# Seed 고정

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) 

In [3]:
# 구글드라이브 활용

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/sample_data/

/content/sample_data


In [5]:
!unzip -qq '/content/drive/MyDrive/project_study_KKUL/008_LGaimers/LG_DATA.zip'

In [6]:
test = pd.read_csv('/content/sample_data/test.csv')
train = pd.read_csv('/content/sample_data/train.csv')
sub = pd.read_csv('/content/sample_data/sample_submission.csv')

# 2.데이터 확인

## 1) train data 확인

* 598 rows x 2881 columns

In [7]:
train.info

<bound method DataFrame.info of     PRODUCT_ID  Y_Class  Y_Quality         TIMESTAMP     LINE PRODUCT_CODE  \
0    TRAIN_000        1   0.533433   2022-06-13 5:14  T050304         A_31   
1    TRAIN_001        2   0.541819   2022-06-13 5:22  T050307         A_31   
2    TRAIN_002        1   0.531267   2022-06-13 5:30  T050304         A_31   
3    TRAIN_003        2   0.537325   2022-06-13 5:39  T050307         A_31   
4    TRAIN_004        1   0.531590   2022-06-13 5:47  T050304         A_31   
..         ...      ...        ...               ...      ...          ...   
593  TRAIN_593        1   0.526546  2022-09-08 14:30  T100306         T_31   
594  TRAIN_594        0   0.524022  2022-09-08 22:38  T050304         A_31   
595  TRAIN_595        0   0.521289  2022-09-08 22:47  T050304         A_31   
596  TRAIN_596        1   0.531375  2022-09-08 14:38  T100304         O_31   
597  TRAIN_597        1   0.533702  2022-09-08 14:46  T100306         O_31   

      X_1   X_2  X_3   X_4  ...

In [ ]:
train.head()

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,A_31,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,A_31,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN


### LINE, PRODUCT_CODE 별 데이터 확인

In [ ]:
cols = ['LINE', 'PRODUCT_CODE']

for col in cols:
  def count_into_list(lst):
    answer = dict()
    for num in train[col]:
      if num not in answer.keys():
        answer[num] = 1
      else:
        answer[num] += 1
    return answer

  count_train = count_into_list(train[col])
  print(f'-- {col} --')
  print(count_train, '\n')

-- LINE --
{'T050304': 78, 'T050307': 42, 'T100304': 175, 'T100306': 174, 'T010306': 70, 'T010305': 59} 

-- PRODUCT_CODE --
{'A_31': 249, 'T_31': 343, 'O_31': 6} 



### PRODUCT_CODE 별로 결측치 확인
* PRODUCT_CODE 별로 결측치가 각기 다른 열에 존재함을 확인할 수 있음

In [ ]:
train[train['PRODUCT_CODE'] == 'A_31'].head()

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,A_31,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,A_31,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN


In [ ]:
train[train['PRODUCT_CODE'] == 'T_31'].head()

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
22,TRAIN_022,0,0.517719,2022-06-14 8:53,T100304,T_31,2.0,102.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,TRAIN_023,0,0.519090,2022-06-14 9:01,T100304,T_31,2.0,102.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,TRAIN_025,1,0.529362,2022-06-19 9:11,T100304,T_31,2.0,97.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,TRAIN_026,1,0.531992,2022-06-19 9:20,T100306,T_31,2.0,95.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,TRAIN_029,1,0.532405,2022-06-19 23:31,T100304,T_31,2.0,100.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### PIVOT
* PRODUCT_CODE, LINE을 기준으로 PIVOT Table 만들기
* 이때, PRODUCT_CODE 별로 해당하는 라인이 각각 다름을 확인할 수 있음

In [ ]:
train_pivot = pd.pivot_table(train, index = 'PRODUCT_CODE', columns = 'LINE', values = 'Y_Class', aggfunc = 'count')
train_pivot

LINE,T010305,T010306,T050304,T050307,T100304,T100306
PRODUCT_CODE,,,,,,
A_31,59.0,70.0,78.0,42.0,NaN,NaN
O_31,NaN,NaN,NaN,NaN,3.0,3.0
T_31,NaN,NaN,NaN,NaN,172.0,171.0


## 2) test data 확인

In [ ]:
test.head()

,PRODUCT_ID,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TEST_000,2022-09-09 2:01,T100306,T_31,2.0,94.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TEST_001,2022-09-09 2:09,T100304,T_31,2.0,93.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TEST_002,2022-09-09 8:42,T100304,T_31,2.0,95.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TEST_003,2022-09-09 10:56,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TEST_004,2022-09-09 11:04,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### LINE, PRODUCT_CODE 별 데이터 확인

In [ ]:
cols = ['LINE', 'PRODUCT_CODE']

for col in cols:
  def count_into_list(lst):
    answer = dict()
    for num in test[col]:
      if num not in answer.keys():
        answer[num] = 1
      else:
        answer[num] += 1
    return answer
    
  count_test = count_into_list(test[col])
  print(f'-- {col} --')
  print(count_test, '\n')

-- LINE --
{'T100306': 132, 'T100304': 111, 'T010305': 14, 'T010306': 14, 'T050304': 13, 'T050307': 26} 

-- PRODUCT_CODE --
{'T_31': 239, 'A_31': 67, 'O_31': 4} 



### PRODUCT_CODE 별로 결측치 확인
* PRODUCT_CODE 별로 결측치가 각기 다른 열에 존재함을 확인할 수 있음

In [ ]:
test[test['PRODUCT_CODE'] == 'A_31'].head()

,PRODUCT_ID,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
3,TEST_003,2022-09-09 10:56,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TEST_004,2022-09-09 11:04,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,TEST_005,2022-09-09 19:35,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,TEST_006,2022-09-09 19:43,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,TEST_007,2022-09-10 12:27,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,57.74,52.51,54.45,57.99,63.16,1.0,NaN,NaN,NaN,NaN


In [ ]:
test[test['PRODUCT_CODE'] == 'T_31'].head()

,PRODUCT_ID,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TEST_000,2022-09-09 2:01,T100306,T_31,2.0,94.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TEST_001,2022-09-09 2:09,T100304,T_31,2.0,93.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TEST_002,2022-09-09 8:42,T100304,T_31,2.0,95.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,TEST_015,2022-09-20 5:20,T100304,T_31,2.0,102.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,TEST_016,2022-09-20 5:28,T100306,T_31,2.0,93.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 3.데이터 전처리

## 1) 불필요한 열 삭제

In [9]:
train_x = train.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_y = train['Y_Class']

test_x = test.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

## 2) Label Encoding

In [10]:
# 라벨 인코딩 전 train data
train_x['PRODUCT_CODE'].unique()

array(['A_31', 'T_31', 'O_31'], dtype=object)

In [11]:
# 라벨 인코딩 전 test data
test_x['PRODUCT_CODE'].unique()

array(['T_31', 'A_31', 'O_31'], dtype=object)

In [12]:
# qualitative to quantitative

qual_col = ['PRODUCT_CODE', 'LINE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test[i]) 
print('Done.')

Done.


In [13]:
# 라벨 인코딩 후 train data
train_x['PRODUCT_CODE'].unique()

array([0, 2, 1])

In [14]:
# 라벨 인코딩 후 test data
test_x['PRODUCT_CODE'].unique()

array([2, 0, 1])

## 3) 정규화

In [ ]:
# 정규화 전 train data
train_x.loc[:3, 'X_2866':'X_2870']

,X_2866,X_2867,X_2868,X_2869,X_2870
0,39.34,40.89,32.56,34.09,77.77
1,38.89,42.82,43.92,35.34,72.55
2,39.19,36.65,42.47,36.53,78.35
3,37.74,39.17,52.17,30.58,71.78


In [ ]:
# 정규화 전 test data
test_x.loc[:3, 'X_1':'X_5']

,X_1,X_2,X_3,X_4,X_5
0,2.0,94.0,0.0,45.0,10.0
1,2.0,93.0,0.0,45.0,11.0
2,2.0,95.0,0.0,45.0,11.0
3,NaN,NaN,NaN,NaN,NaN


In [ ]:
from sklearn.preprocessing import MinMaxScaler

x_col = train_x.columns[train_x.columns.str.contains('X')].tolist()
scaler = MinMaxScaler()
  
scaler.fit(train_x[x_col])
  
train_x[x_col] = scaler.transform(train_x[x_col])
test_x[x_col] = scaler.transform(test_x[x_col])

In [ ]:
# 정규화 후 train data
train_x.loc[:3, 'X_2866':'X_2870']

,X_2866,X_2867,X_2868,X_2869,X_2870
0,0.256757,0.248647,0.000000,0.122283,0.890487
1,0.240754,0.300866,0.407899,0.164742,0.601770
2,0.251422,0.133929,0.355835,0.205163,0.922566
3,0.199858,0.202110,0.704129,0.003057,0.559181


In [ ]:
# 정규화 후 test data
test_x.loc[:3, 'X_1':'X_5']

,X_1,X_2,X_3,X_4,X_5
0,0.009804,0.466667,0.0,0.0,0.0
1,0.009804,0.400000,0.0,0.0,1.0
2,0.009804,0.533333,0.0,0.0,1.0
3,NaN,NaN,NaN,NaN,NaN


## 4) 결측값 'train_x' 평균으로 채우기

In [ ]:
# 결측값 대체 전 train data
train_x.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.256757,0.248647,0.000000,0.122283,0.890487,NaN,NaN,NaN,NaN,NaN
1,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.240754,0.300866,0.407899,0.164742,0.601770,NaN,NaN,NaN,NaN,NaN
2,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.251422,0.133929,0.355835,0.205163,0.922566,NaN,NaN,NaN,NaN,NaN
3,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.199858,0.202110,0.704129,0.003057,0.559181,NaN,NaN,NaN,NaN,NaN
4,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.233997,0.275703,0.515978,0.088315,0.846239,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 결측값 대체 전 test data
test_x.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,5,2,0.009804,0.466667,0.0,0.0,0.0,0.0,0.352941,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,2,0.009804,0.400000,0.0,0.0,1.0,0.0,0.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,2,0.009804,0.533333,0.0,0.0,1.0,0.0,0.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# data leakage를 방지하기 위해 train_x의 평균으로 test_x의 결측치를 채움

train_x = train_x.fillna(train_x.mean())
test_x = test_x.fillna(train_x.mean())

In [ ]:
# 결측값 대체 후 train data
train_x.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.256757,0.248647,0.000000,0.122283,0.890487,0.0,NaN,NaN,NaN,NaN
1,3,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.240754,0.300866,0.407899,0.164742,0.601770,0.0,NaN,NaN,NaN,NaN
2,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.251422,0.133929,0.355835,0.205163,0.922566,0.0,NaN,NaN,NaN,NaN
3,3,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.199858,0.202110,0.704129,0.003057,0.559181,0.0,NaN,NaN,NaN,NaN
4,2,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.233997,0.275703,0.515978,0.088315,0.846239,0.0,NaN,NaN,NaN,NaN


In [ ]:
# 결측값 대체 후 test data
test_x.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,5,2,0.009804,0.466667,0.0,0.0,0.00000,0.0,0.352941,0.000000,...,0.664555,0.592741,0.612072,0.719083,0.275426,0.0,NaN,NaN,NaN,NaN
1,4,2,0.009804,0.400000,0.0,0.0,1.00000,0.0,0.000000,0.000000,...,0.664555,0.592741,0.612072,0.719083,0.275426,0.0,NaN,NaN,NaN,NaN
2,4,2,0.009804,0.533333,0.0,0.0,1.00000,0.0,0.000000,0.000000,...,0.664555,0.592741,0.612072,0.719083,0.275426,0.0,NaN,NaN,NaN,NaN
3,0,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.664555,0.592741,0.612072,0.719083,0.275426,0.0,NaN,NaN,NaN,NaN
4,1,0,0.013821,0.541547,0.0,0.0,0.39255,0.0,0.223664,0.048711,...,0.664555,0.592741,0.612072,0.719083,0.275426,0.0,NaN,NaN,NaN,NaN


## 5) 결측값이 있는 x 변수 Drop

In [ ]:
# 결측값 존재 여부 확인 _ train data
train_x.isnull().sum()

LINE              0
PRODUCT_CODE      0
X_1               0
X_2               0
X_3               0
               ... 
X_2871            0
X_2872          598
X_2873          598
X_2874          598
X_2875          598
Length: 2877, dtype: int64

In [ ]:
# 결측값 존재 여부 확인 _ test data
test_x.isnull().sum()

LINE              0
PRODUCT_CODE      0
X_1               0
X_2               0
X_3               0
               ... 
X_2871            0
X_2872          310
X_2873          310
X_2874          310
X_2875          310
Length: 2877, dtype: int64

In [ ]:
train_x = train_x.dropna(axis =1)
test_x = test_x.dropna(axis=1)

In [ ]:
# 결측값 존재 여부 확인 _ train data
train_x.isnull().sum()

LINE            0
PRODUCT_CODE    0
X_1             0
X_2             0
X_3             0
               ..
X_2867          0
X_2868          0
X_2869          0
X_2870          0
X_2871          0
Length: 2795, dtype: int64

In [ ]:
# 결측값 존재 여부 확인 _ test data
test_x.isnull().sum()

LINE            0
PRODUCT_CODE    0
X_1             0
X_2             0
X_3             0
               ..
X_2867          0
X_2868          0
X_2869          0
X_2870          0
X_2871          0
Length: 2795, dtype: int64

# 4.Modeling

In [ ]:
# train_x, train_y, test_x 형태 확인
train_x.shape, train_y.shape, test_x.shape

((598, 2795), (598,), (310, 2795))

In [ ]:
# lightGBM, ExtraTreesClassifier, RandomForestClassifier 선택

model = lgbm.LGBMClassifier()
et_cls = ExtraTreesClassifier(n_estimators= 500, min_samples_leaf = 5, min_samples_split= 7, max_features=2795)
rf_cls = RandomForestClassifier(n_estimators=500, min_samples_leaf=5, min_samples_split=7, max_features=2795)
lg_cls = model.fit(train_x, train_y)
et_cls.fit(train_x, train_y)

# 모델 voting

voting = VotingClassifier(
    estimators=[
        ('et', et_cls),
        ('rf', rf_cls),
        ('lg', lg_cls)
    ]
)
voting.fit(train_x, train_y)

VotingClassifier(estimators=[('et',
                              ExtraTreesClassifier(max_features=2795,
                                                   min_samples_leaf=5,
                                                   min_samples_split=7,
                                                   n_estimators=500)),
                             ('rf',
                              RandomForestClassifier(max_features=2795,
                                                     min_samples_leaf=5,
                                                     min_samples_split=7,
                                                     n_estimators=500)),
                             ('lg', LGBMClassifier())])

# 5.Predict

In [ ]:
pred = et_cls.predict(test_x)
sub['Y_Class'] = pred

In [ ]:
sub.head()

,PRODUCT_ID,Y_Class
0,TEST_000,1
1,TEST_001,1
2,TEST_002,1
3,TEST_003,1
4,TEST_004,1


In [ ]:
# pred 값 확인
pred

array([1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 2, 1, 2, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
# 저장
sub.to_csv("/content/sample_data/answer.csv", index = False)